In [1]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained BERT model for question answering
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

/Users/robinviltoriano/Library/CloudStorage/Box-Box/Trimester 1 2024/NLP/NLP_Assignment_2/nlp_assignment_2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
# Prepare input text
question = "What is the capital of France?"
context = "The capital of France is Paris."

# Tokenize input text
inputs = tokenizer.encode_plus(question, context, return_tensors="pt", max_length=512, truncation=True)

# Perform question answering
model_output = model(**inputs)
start_scores = model_output['start_logits']
end_scores = model_output['end_logits']

# Get answer span
start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores)

# Convert token indices to actual tokens
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

if end_index >= start_index:
    answer_tokens = tokens[start_index:end_index]
else:
    answer_tokens = tokens[end_index:start_index]

# Convert answer tokens to string
answer = tokenizer.convert_tokens_to_string(answer_tokens)

print("Answer:", answer)


Answer: paris .
